<a href="https://colab.research.google.com/github/DariaKarelina/web-course/blob/main/cript_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gmpy2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.0 MB/s eta 0:00:00


In [ ]:
#ГЕНЕРАЦИЯ КЛЮЧЕЙ
import random
import math
from gmpy2 import is_prime, mpz

def generate_large_prime(l):
    while True:
        # генерируем случайное число из l знаков
        lower = 10**(l-1)
        upper = 10**l - 1
        candidate = random.randint(lower, upper)

        # проверка на простоту
        if is_prime(candidate):
            return candidate


def generate_rsa_keys_with_primes():
    # генерирую два простых числа пэ и ку
    p = generate_large_prime(150)
    q = generate_large_prime(150)
    print(f"\nСгенерированные простые числа:")
    print(f"p = {p}")
    print(f"q = {q}\n")

    n = p * q

    # функция эйлера от н, (такая, тк p и q -- простые)
    phi = (p - 1) * (q - 1)

    # открытая экспонента (должна быть взаимнопроста с фи от н)
    e = generate_large_prime(300)

    # секретная экспонента d (обратная к е по модулю фи от н)
    d = modinv(e, phi)
    print(d)
    # Возвращаем ключи и простые числа
    public_key = (n, e)
    private_key = (n, d)
    return public_key, private_key, p, q

def extended_gcd(a, b):
    """Расширенный алгоритм Евклида для нахождения НОД и коэффициентов"""
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = extended_gcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = extended_gcd(a, m)
    if g != 1:
        raise Exception('Модульного обратного не существует')
    else:
        return x % m

# Пример использования
if __name__ == "__main__":
    public_key, private_key, p, q = generate_rsa_keys_with_primes()

    print(f"Открытый ключ (N, e): {public_key}")
    print(f"Закрытый ключ (N, d): ({private_key[0]}, ...)")


Сгенерированные простые числа:
p = 503332246690457252044356634877409950813798521602557622227828680812095888514829630248103672159671757225770452987662576636489971995064530997732114408911
q = 257913286337937375213841009673110989332068014911681805836107501624557722017255777464780138832907917153887467892875660652592214549230272986715490434481

59614640173357769419728290453196092412738357420419585786901506725718862117106564503918097593657868594423806827994762905534460965559809080380890145639661873781276961449534945442459371668174247141602879518534403728914327383021842069495770261973330343448054928524372385316098559062154103665105832360977
Открытый ключ (N, e): (129816073863793233024348874596041028318869249768717974118651563326160799882069454894351968084358127450616746321201790946813054622885941635534854382685758252509699643932036760373926657944957848695783498430783906909264243969927626069125781230756458309843420361602101504547206330372403942324531888060191, 43913526747774890743648604690955

In [ ]:
# ШИФРОВАНИЕ И ДЕШИФРОВАНИЕ
import binascii

#n = 129816073863793233024348874596041028318869249768717974118651563326160799882069454894351968084358127450616746321201790946813054622885941635534854382685758252509699643932036760373926657944957848695783498430783906909264243969927626069125781230756458309843420361602101504547206330372403942324531888060191
n = 367506764883191975594831294623114043774451938672254139901485316914325686901020506115198901916775190845329504378231824789189850048341376695164449551430689280963883751091427559794513745387629409376944511547113286948273653297483509953852175641605117067051638804105095109138579565404381933238621355499638632995837
e = 220687473778277042990256271210769033685985618807141639522542153516158065121900992759170705693150934206691083725179645347854293191889170862908390244501927709038306364454158328734300467660903834931990569267543119182349533579467946921554227350447240357794903257869444076709124780963220027882623586524053013368573
#e = 439135267477748907436486046909555404809549592539914777103755928622217914488990002828812709637974910063309567387946952133027180770165941417802181174854924042066273439236449137963854653231085029097907110518503875886983498446302737617404646025396280609457186667868182638083754742907419699349565949098513
d = 59614640173357769419728290453196092412738357420419585786901506725718862117106564503918097593657868594423806827994762905534460965559809080380890145639661873781276961449534945442459371668174247141602879518534403728914327383021842069495770261973330343448054928524372385316098559062154103665105832360977

public_key = (n, e)
private_key = (n, d)

def encrypt_message(message, public_key):
    """Шифрование сообщения с использованием открытого ключа"""
    n, e = public_key

    # Проверяем длину сообщения
    if len(message) > 64:
        raise ValueError("Сообщение должно быть не длиннее 64 символов")

    # Преобразуем сообщение в бинарную строку через ASCII
    binary_str = bin(int(binascii.hexlify(message.encode('utf-8')), 16))[2:]

    # Преобразуем бинарную строку в число
    message_num = int(binary_str, 2)

    # Проверяем, что число меньше N
    if message_num >= n:
        raise ValueError("Сообщение слишком длинное для данного ключа")

    # Шифруем: c = m^e mod n
    encrypted_num = pow(message_num, e, n)
    return encrypted_num

def decrypt_message(encrypted_num, private_key):
    """Дешифрование сообщения с использованием закрытого ключа"""
    n, d = private_key

    # Дешифруем: m = c^d mod n
    decrypted_num = pow(encrypted_num, d, n)

    # Преобразуем число обратно в бинарную строку
    binary_str = bin(decrypted_num)[2:]

    # Дополняем ведущими нулями до байтовой границы (если нужно)
    if len(binary_str) % 8 != 0:
        binary_str = binary_str.zfill(8 * ((len(binary_str) // 8 + 1)))

    # Преобразуем бинарную строку в байты, затем в строку
    try:
        # Сначала преобразуем число в hex строку
        hex_str = hex(decrypted_num)[2:]  # [2:] чтобы убрать '0x'
        # Если длина hex строки нечетная, добавляем ведущий ноль
        if len(hex_str) % 2 != 0:
            hex_str = '0' + hex_str
        decrypted_bytes = binascii.unhexlify(hex_str)
        decrypted_message = decrypted_bytes.decode('utf-8')
    except (binascii.Error, UnicodeDecodeError) as e:
        raise ValueError("Ошибка дешифрования: некорректные данные") from e

    return decrypted_message

# Пример использования
if __name__ == "__main__":
    message = "Allo! Do you listening me?"
    print(f"Исходное сообщение: {message}")
    try:
        # Шифрование
        encrypted = encrypt_message(message, public_key)
        print(f"шифр: {encrypted}")
        encrypt = 123694684550998438239107746274247308782559525311795134493077744644776578440195826627741871098189569351093928565413659332709015322683066619635689640783442694434521012214648731434270816220857550405623106194944104233720816684758658477594980339023496423056420749040584932540166043104508194819626251648027
        # Дешифрование
        decrypted = decrypt_message(encrypt, private_key)
        print(f"сообщение: {decrypted}")
    except ValueError as e:
        print(f"Ошибка: {e}")

Исходное сообщение: Allo! Do you listening me?
шифр: 259820896611008339126293027190829844535459311360519882758204441045178933818502943104358177297477595024405728974827825764433405531900307775520599741090431117218260257908228998706523771433067747603630379619640407163367510217954602305542137170463466992768608829974782491782632799703716208914313111635680912802352
Ошибка: Ошибка дешифрования: некорректные данные
